In [3]:
!pip install librosa
!pip install sagemaker --upgrade

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [4]:
import json
import time
import boto3
import numpy as np
import sagemaker
import sagemaker.huggingface

#BUCKET="[BUCKET_NAME]" # please use your bucket name
ROLE = sagemaker.get_execution_role()
sess = sagemaker.Session()
BUCKET = sess.default_bucket()
print(f"sagemaker role arn: {ROLE}")
print(f"sagemaker bucket: {BUCKET}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::348052051973:role/service-role/AmazonSageMakerServiceCatalogProductsExecutionRole
sagemaker bucket: sagemaker-us-east-1-348052051973
sagemaker session region: us-east-1


In [26]:
from sagemaker.huggingface import HuggingFaceModel

model_name = 'whisper-zhcn4'
model_data = 's3://sagemaker-us-east-1-348052051973/whisper-zhcn/whisper-zhcn-1672987430/output/whisper-zhcn-1672987430/output/model.tar.gz'
instance_type = 'ml.m5d.4xlarge'
huggingface_model = HuggingFaceModel(
        entry_point = 'inference2.py',
        source_dir='./scripts',
        name = model_name,
        transformers_version='4.17.0',
        pytorch_version='1.10.2',
        py_version='py38',
        model_data=model_data,
        role=ROLE,
    )

In [27]:
# predictor = huggingface_model.deploy(
#     initial_instance_count=1,
#     instance_type="ml.m5d.4xlarge", 
#     endpoint_name = model_name,
# )

In [ ]:
# first time deploy as endpoint
# predictor = huggingface_model.deploy(
#     initial_instance_count=1,
#     instance_type="ml.m5d.2xlarge", 
#     endpoint_name = model_name,
# )

# update endpoint
from sagemaker.predictor import Predictor
from sagemaker.serializers import DataSerializer
from sagemaker.deserializers import JSONDeserializer

sess.create_model(
    model_name,
    ROLE,
    huggingface_model.prepare_container_def(
        instance_type=instance_type
    )
)
audio_serializer = DataSerializer(content_type='audio/x-audio')


predictor = Predictor('whisper-zhcn', serializer=audio_serializer, deserializer=JSONDeserializer())
predictor.update_endpoint(model_name=model_name, initial_instance_count=1, instance_type=instance_type)

In [ ]:
# # inference audio file that download from S3 bucket or inference local audio file 
# import soundfile
# import IPython.display as ipd
# import numpy as np

# # s3.download_file(BUCKET, 'huggingface-blog/sample_audio/xxxxxx.wav', 'downloaded.wav')
# # file_name ='downloaded.wav'

# # download a sample audio file by using below link
# !wget https://datashare.ed.ac.uk/bitstream/handle/10283/343/MKH800_19_0001.wav
    
# file_name ='MKH800_19_0001.wav'

# speech_array, sampling_rate = soundfile.read(file_name)

# ipd.Audio(data=np.asarray(speech_array), autoplay=False, rate=16000)

In [22]:
%%time
from sagemaker.predictor import Predictor
from sagemaker.serializers import DataSerializer

audio_path = 'audio.mp3'
audio_serializer = DataSerializer(content_type='audio/x-audio')

predictor = Predictor('whisper-zhcn', serializer=audio_serializer)
with open(audio_path, "rb") as data_file:
    audio_data = data_file.read()
    
prediction = predictor.predict(audio_data)
print(prediction)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from primary with message "{
  "code": 400,
  "type": "InternalServerException",
  "message": "We expect a numpy ndarray as input, got `\u003cclass \u0027bytearray\u0027\u003e`"
}
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/whisper-zhcn in account 348052051973 for more information.

In [ ]:
import sys
sys.getsizeof(speech_array) 

In [2]:
!df -kh


Filesystem      Size  Used Avail Use% Mounted on
devtmpfs        1.9G     0  1.9G   0% /dev
tmpfs           1.9G     0  1.9G   0% /dev/shm
tmpfs           1.9G  584K  1.9G   1% /run
tmpfs           1.9G     0  1.9G   0% /sys/fs/cgroup
/dev/nvme0n1p1  140G   51G   90G  37% /
/dev/nvme1n1    985G  524K  935G   1% /home/ec2-user/SageMaker
tmpfs           387M     0  387M   0% /run/user/1001
tmpfs           387M     0  387M   0% /run/user/1002
tmpfs           387M     0  387M   0% /run/user/1000


In [1]:
!sudo find /tmp -type f -delete
